This is a sample for celestial navigation for a stationary observer<br>
© August Linnman, 2025, email: august@linnman.net<br>
MIT License (see [LICENSE file](LICENSE))

Jupyter notebook for navigation on a moving ship/vehicle. 

In [16]:
# Importing Python libraries
from time import time
from starfix import Sight, SightTrip, get_representation, LatLonGeodetic,\
                    IntersectError

In [ ]:
# Input form
import ipywidgets as widgets
from ipywidgets import VBox
import json

file_name = "notebook-sea-dict.json"

try:
    with open(file_name, "r") as f:
        s = f.read ()
        numDict = json.loads (s)
except FileNotFoundError:
    numDict = {"ObjectName1" : "Sun",
               "Altitude1" : "30:51:27.1",
               "Time1" : "2024-06-20 06:14:38+00:00",
               
               "ObjectName2" : "Sun",
               "Altitude2" : "38:34:21.6",
               "Time2" : "2024-06-20 07:14:38+00:00",

               "Course" : "175",
               "Speed"  : "20",
               
               "DrpLat" : "59",
               "DrpLon" : "18"}

def dump_dict ():
    jDump = json.dumps (numDict)
    with open(file_name, "w") as f:
        f.write(jDump)    

def handle_change (change):
    if change['type'] == 'change' and change['name'] == 'value':
        the_owner = change['owner']
        assert isinstance (the_owner, TextWidget)
        the_owner.handle_event (change)

class TextWidget (widgets.Text):

    def __init__ (self, attrName, description):
        self.__attrName = attrName
        super().__init__ (numDict[self.__attrName], description=description, disabled=False)
        self.observe (handle_change)

    def handle_event (self, change):
        numDict[self.__attrName] =change['new']
        dump_dict ()

widget_array = []
typeArray = ["ObjectName", "Altitude", "Time"]
labelArray = ["NAME", "ALT", "TIME"]
widget_array.append (TextWidget ("DrpLat","DRP_LAT"))
widget_array.append (TextWidget ("DrpLon","DRP_LON"))
for i in range (2):
    for j in range (3):
        widget_array.append (TextWidget (typeArray[j]+str(i+1),
                                         description=labelArray[j]+"_"+str(i+1)))
widget_array.append (TextWidget ("Course","COURSE"))
widget_array.append (TextWidget ("Speed","SPEED"))       

VBox (widget_array)

In [18]:
# Estimated (DR) position at start
s1LatLon = LatLonGeodetic (float(numDict["DrpLat"]), float(numDict["DrpLon"]))
# Estimated course and speed
C_COURSE = float(numDict["Course"])
SPEED = float(numDict["Speed"])

#This is the star fix for s1, the starting point

Sight.set_estimated_position (s1LatLon)

s1 = Sight (  object_name          = numDict["ObjectName1"],
              set_time             = numDict["Time1"],
              measured_alt         = numDict["Altitude1"]
              )

In [19]:
# Star fix for ending point

s2 = Sight (  object_name          = numDict["ObjectName2"],
              set_time             = numDict["Time2"],
              measured_alt         = numDict["Altitude2"]
              )

In [ ]:
# Sight reduction

st = SightTrip (sight_start               = s1,\
                 sight_end                = s2,\
                 estimated_starting_point = s1LatLon,\
                 course_degrees           = C_COURSE,\
                 speed_knots              = SPEED)
m = None
try:
    intersections, _, _ = st.get_intersections (return_geodetic=True)
    assert isinstance (intersections, tuple)
    print ("Starting point = " + str(get_representation(intersections[1],1)))
    print ("End point = " + str(get_representation(intersections[0],1)))
    m = st.render_folium(intersections)
except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve)) 
m